# 🤝 Multi-Agent Coordination

> **Build trust and coordinate between agents using IATP.**

## Learning Objectives

By the end of this notebook, you will:
1. Understand the Inter-Agent Trust Protocol (IATP)
2. Sign and verify agent messages cryptographically
3. Establish trust relationships between agents
4. Coordinate multi-agent workflows securely
5. Handle trust violations and revocations

---

## Why Multi-Agent Trust?

**Problem:** When agents communicate, how does Agent B know Agent A is legitimate?

**Solution:** Cryptographic message signing. Every message carries proof of origin.

```
Without Trust Protocol:      With IATP:

  Agent A                      Agent A
     ↓                            ↓
 "Do X" (unsigned)           "Do X" + signature
     ↓                            ↓
  Agent B                      Agent B
     ↓                            ↓
  Trust it? 🤷               Verify signature ✓
```

---

## Step 1: Install Dependencies

In [ ]:
!pip install agent-os[iatp] --quiet

## Step 2: Create Agent Identities

In [ ]:
from iatp import AgentIdentity, TrustRegistry

# Create identities for two agents
alice = AgentIdentity.create(
    agent_id="alice-001",
    name="Alice the Analyst",
    capabilities=["data_analysis", "report_generation"]
)

bob = AgentIdentity.create(
    agent_id="bob-001",
    name="Bob the Builder",
    capabilities=["code_generation", "testing"]
)

print("🆔 Agent Identities Created")
print("=" * 50)
print(f"\nAlice:")
print(f"  ID: {alice.agent_id}")
print(f"  Public Key: {alice.public_key[:40]}...")
print(f"  Capabilities: {alice.capabilities}")

print(f"\nBob:")
print(f"  ID: {bob.agent_id}")
print(f"  Public Key: {bob.public_key[:40]}...")
print(f"  Capabilities: {bob.capabilities}")

## Step 3: Sign Messages

In [ ]:
from iatp import SignedMessage

# Alice creates and signs a message
message = SignedMessage.create(
    sender=alice,
    recipient_id="bob-001",
    content={
        "action": "generate_report",
        "data_source": "sales_q4.csv",
        "format": "pdf"
    }
)

print("✉️  Signed Message Created")
print("=" * 50)
print(f"From: {message.sender_id}")
print(f"To: {message.recipient_id}")
print(f"Content: {message.content}")
print(f"Signature: {message.signature[:60]}...")
print(f"Timestamp: {message.timestamp}")

## Step 4: Verify Messages

In [ ]:
# Bob receives and verifies the message
registry = TrustRegistry()
registry.register(alice)  # Alice's public key is registered

# Verify the message
verification = registry.verify(message)

print("🔍 Message Verification")
print("=" * 50)
print(f"Valid Signature: {verification.is_valid}")
print(f"Sender Verified: {verification.sender_verified}")
print(f"Timestamp Valid: {verification.timestamp_valid}")

if verification.is_valid:
    print(f"\n✅ Bob can trust this message came from Alice!")

In [ ]:
# What happens with a tampered message?
tampered = message.copy()
tampered.content["data_source"] = "secret_passwords.txt"  # Attacker changes content

verification = registry.verify(tampered)

print("🔍 Tampered Message Verification")
print("=" * 50)
print(f"Valid Signature: {verification.is_valid}")
print(f"\n🚨 Tampering detected! Signature doesn't match content.")

## Step 5: Establish Trust Relationships

In [ ]:
from iatp import TrustLevel

# Create a trust registry
registry = TrustRegistry()

# Register agents with different trust levels
registry.register(alice, trust_level=TrustLevel.HIGH)
registry.register(bob, trust_level=TrustLevel.MEDIUM)

# Create a new untrusted agent
mallory = AgentIdentity.create(
    agent_id="mallory-001",
    name="Mallory the Malicious",
    capabilities=["hacking", "deception"]
)
registry.register(mallory, trust_level=TrustLevel.NONE)

print("🏛️  Trust Registry")
print("=" * 50)
for agent_id, info in registry.list_agents().items():
    trust = info['trust_level']
    emoji = "🟢" if trust == TrustLevel.HIGH else "🟡" if trust == TrustLevel.MEDIUM else "🔴"
    print(f"{emoji} {agent_id}: {trust.name}")

In [ ]:
# Check if agents can communicate
can_alice_to_bob = registry.can_communicate("alice-001", "bob-001")
can_mallory_to_alice = registry.can_communicate("mallory-001", "alice-001")

print("🔗 Communication Permissions")
print("=" * 50)
print(f"Alice → Bob: {'✅ Allowed' if can_alice_to_bob else '❌ Denied'}")
print(f"Mallory → Alice: {'✅ Allowed' if can_mallory_to_alice else '❌ Denied'}")

## Step 6: Multi-Agent Workflow

In [ ]:
from agent_os import KernelSpace
from iatp import AgentIdentity, TrustRegistry, SignedMessage

# Initialize
kernel = KernelSpace(policy="strict")
registry = TrustRegistry()

# Create coordinator and worker agents
coordinator = AgentIdentity.create("coordinator", "Task Coordinator", ["orchestration"])
worker_1 = AgentIdentity.create("worker-1", "Data Worker", ["data_processing"])
worker_2 = AgentIdentity.create("worker-2", "ML Worker", ["model_training"])

registry.register(coordinator, TrustLevel.HIGH)
registry.register(worker_1, TrustLevel.MEDIUM)
registry.register(worker_2, TrustLevel.MEDIUM)

@kernel.register
async def coordinator_agent(task: str):
    """Coordinator that delegates to workers."""
    results = []
    
    # Send signed task to Worker 1
    msg1 = SignedMessage.create(
        sender=coordinator,
        recipient_id="worker-1",
        content={"action": "process_data", "task": task}
    )
    results.append(f"Sent to worker-1: {msg1.content['action']}")
    
    # Send signed task to Worker 2
    msg2 = SignedMessage.create(
        sender=coordinator,
        recipient_id="worker-2",
        content={"action": "train_model", "task": task}
    )
    results.append(f"Sent to worker-2: {msg2.content['action']}")
    
    return {"tasks_delegated": 2, "messages": results}

@kernel.register
async def worker_agent(message: SignedMessage):
    """Worker that verifies and processes messages."""
    # Verify the message
    verification = registry.verify(message)
    
    if not verification.is_valid:
        return {"error": "Invalid message signature", "processed": False}
    
    # Check trust level of sender
    sender_trust = registry.get_trust_level(message.sender_id)
    if sender_trust.value < TrustLevel.MEDIUM.value:
        return {"error": "Sender not trusted", "processed": False}
    
    # Process the task
    return {
        "processed": True,
        "action": message.content["action"],
        "sender_verified": True,
        "trust_level": sender_trust.name
    }

# Execute coordinator
result = await kernel.execute(coordinator_agent, "Analyze sales data and train predictor")

print("🎯 Coordinator Result")
print("=" * 50)
for k, v in result.items():
    print(f"  {k}: {v}")

## Step 7: Trust Revocation

In [ ]:
# Simulate a compromised agent
print("🔐 Before Revocation:")
print(f"  Worker-1 trust: {registry.get_trust_level('worker-1').name}")

# Revoke trust
registry.revoke("worker-1", reason="Suspicious activity detected")

print(f"\n🚨 After Revocation:")
print(f"  Worker-1 trust: {registry.get_trust_level('worker-1').name}")

# Try to send message from revoked agent
suspicious_msg = SignedMessage.create(
    sender=worker_1,
    recipient_id="coordinator",
    content={"action": "steal_data"}
)

verification = registry.verify(suspicious_msg)
print(f"\n🔍 Message from revoked agent:")
print(f"  Valid: {verification.is_valid}")
print(f"  Reason: {verification.rejection_reason}")

## Step 8: Message Bus Integration

In [ ]:
from amb_core import MessageBus, Topic
from iatp import SignedMessage

# Create a message bus
bus = MessageBus()

# Define topics
tasks_topic = Topic("agent.tasks")
results_topic = Topic("agent.results")

# Subscribe workers to tasks
async def handle_task(message: SignedMessage):
    # Verify before processing
    if registry.verify(message).is_valid:
        print(f"  ✅ Processing: {message.content}")
    else:
        print(f"  ❌ Rejected: Invalid signature")

bus.subscribe(tasks_topic, handle_task)

# Coordinator publishes signed tasks
task_msg = SignedMessage.create(
    sender=coordinator,
    recipient_id="broadcast",
    content={"action": "analyze", "data": "quarterly_report.csv"}
)

print("📬 Publishing signed task to message bus...")
await bus.publish(tasks_topic, task_msg)

---

## Summary

| Feature | What It Does |
|---------|-------------|
| `AgentIdentity` | Creates cryptographic identity for agents |
| `SignedMessage` | Message with cryptographic signature |
| `TrustRegistry` | Manages agent trust relationships |
| `TrustLevel` | HIGH, MEDIUM, LOW, NONE |
| `verify()` | Validates message authenticity |
| `revoke()` | Removes trust from compromised agent |

### Quick Reference

```python
from iatp import AgentIdentity, SignedMessage, TrustRegistry, TrustLevel

# Create identity
agent = AgentIdentity.create("id", "name", ["capabilities"])

# Sign message
msg = SignedMessage.create(sender=agent, recipient_id="...", content={...})

# Register and verify
registry = TrustRegistry()
registry.register(agent, TrustLevel.HIGH)
result = registry.verify(msg)

# Revoke trust
registry.revoke("agent_id", reason="...")
```

---

## Next Steps

- [06-policy-engine](06-policy-engine.ipynb) - Deep dive into policies
- [IATP Documentation](https://github.com/imran-siddique/iatp)
- [Agent Message Bus Documentation](https://github.com/imran-siddique/amb)